# Converting the queries to flask-sqlalchemy

In [2]:
import sys
import sys
sys.path.append('/Users/shayneufeld/GitHub/parle')
from models import db
from models import *

# get_pol_json 
def get_pol_json(politician_id):
    """
    Returns all the data related to a single specific politician when a user requests their profile.
    Anything we want in the profile has to be sent from here in JSON, so this is where all the magic happens.

    Related React.js component: <ProfileBox />

    :param politician_id: id of the single politician requested
    :return: a jsonify'd dictionary with all the desired profile information
    """
    # Get the cursor
    cursor = get_cursor()

    # Build our select query using data from bills_membervote, bills_votequestion, and bills_bill
    # We are currently only interested in the final vote, so the bill description is used to identify those
    # Add any parameters using (%s) or (%(name)s). More information below.
    query = (
        "SELECT m.votequestion_id,m.vote,b.number,b.name_en,b.law,b.short_title_en, v.session_id "
        "FROM bills_membervote m, bills_votequestion v, bills_bill b "
        "WHERE m.politician_id = (%s) "
        "AND v.id = m.votequestion_id "
        "AND v.description_en LIKE (\'%%Bill be now read a third time and do pass%%\') "
        "AND b.id = v.bill_id "
        "ORDER BY v.date DESC"
    )
    # If any parameters are used in the above query, insert them in the parameters tuple
    # Insert parameters in the order used in the query unless using %(name)s placeholders
    # see http://initd.org/psycopg/docs/usage.html#passing-parameters-to-sql-queries for more information
    parameters = (politician_id,)

    # Execute query
    cursor.execute(query, parameters)

    # Fetch results
    pol_results = cursor.fetchall()

    # return a JSON response to React (includes header, no extra work needed)
    return jsonify(votes=pol_results)

### Retrieve all members

User.query.order_by(User.username)

In [94]:
a = Member.query.order_by(Member.name)

In [101]:
for member in a:
    print(member.name)

Adam Vaughan
Ahmed Hussen
Alain Rayes
Alaina Lockhart
Alexander Nuttall
Alexandra Mendès
Alexandre Boulerice
Ali Ehsassi
Alice Wong
Alistair MacGregor
Alupa Clarke
Amarjeet Sohi
Andrew Leslie
Andrew Scheer
Andy Fillmore
Angelo Iacono
Anita Vandenbeld
Anju Dhillon
Anne Minh-Thu Quach
Anthony Housefather
Anthony Rota
Arif Virani
Arnold Chan
Arnold Viersen
Bardish Chagger
Ben Lobb
Bernadette Jordan
Bernard Généreux
Bev Shipley
Bill Blair
Bill Casey
Bill Morneau
Blaine Calkins
Blake Richards
Bob Bratina
Bob Saroya
Bob Zimmer
Borys Wrzesnewskyj
Brad Trost
Brenda Shanahan
Brian Masse
Brigitte Sansoucy
Bruce Stanton
Bryan May
Candice Bergen
Carla Qualtrough
Carol Hughes
Carolyn Bennett
Cathay Wagantall
Catherine McKenna
Cathy McLeod
Celina Caesar-Chavannes
Chandra Arya
Charlie Angus
Cheryl Gallant
Cheryl Hardcastle
Chris Bittle
Chris Warkentin
Christine Moore
Chrystia Freeland
Colin Carrie
Colin Fraser
Dan Albas
Dan Ruimy
Dan Vandal
Daniel Blaikie
Darrell Samson
Darren Fisher
Darshan Singh Ka

In [3]:
members = Member.query.all()

In [12]:
members[0].url

'/Parliamentarians/en/members/Ziad-Aboultaif(89156)'

In [33]:
'''
in postgres DB              | in new db
                            |
1 member vote_question id   | event.id
2 member vote               | event.id
3 bill number               | bill.id or bill.number?
4 bill name                 | bill.slug
5 bill law                  | bill.number
6 bill title                | bill.short_title
7 vote session id           | event.session_id
'''

TypeError: 'NoneType' object is not callable

In [116]:
events = Event.query.all()

In [119]:
for event in events:
    if '3rd reading' in event.description:
        print(event.description)

3rd reading and adoption of Bill C-9, An Act for granting to Her Majesty certain sums of money for the federal public administration for the fiscal year ending March 31, 20173e lecture et adoption du projet de loi C-9, Loi portant octroi à Sa Majesté de crédits pour l'administration publique fédérale pendant l'exercice se terminant le 31 mars 2017
3rd reading and adoption of Bill C-8, An Act for granting to Her Majesty certain sums of money for the federal public administration for the fiscal year ending March 31, 20163e lecture et adoption du projet de loi C-8, Loi portant octroi à Sa Majesté de crédits pour l'administration publique fédérale pendant l'exercice se terminant le 31 mars 2016
3rd reading and adoption of Bill C-3, An Act for granting to Her Majesty certain sums of money for the federal public administration for the financial year ending March 31, 20163e lecture et adoption du projet de loi C-3, Loi portant octroi à Sa Majesté de crédits pour l'administration publique fédé

In [129]:
events_filtered = Event.query.filter(Event.description.contains('3rd reading'))

In [135]:
for event in events_filtered:
    print(event.bill_id)

9
8
3


In [145]:
votes = Vote.query.join(Event.description,Vote.event_id==Event.id)

AttributeError: mapper

In [132]:
id